In [1]:
import requests
import numpy as np
import pandas as pd

In [2]:
Name = []
Description = []
for i in range(1, 487):
    data = requests.get(f"https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}")
    data = data.json()['results']
    for movie in data:
        Name.append(movie['title'])
        Description.append(movie['overview'])

In [3]:
df = pd.DataFrame({
    'Name': Name,
    'Description': Description
})
df.head()

,Name,Description
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,In the continuing saga of the Corleone crime f...
3,Schindler's List,The true story of how businessman Oskar Schind...
4,12 Angry Men,The defense and the prosecution have rested an...


## Lower Casing

In [6]:
df['Description'] = df['Description'].str.lower()
df.head()

,Name,Description
0,The Shawshank Redemption,imprisoned in the 1940s for the double murder ...
1,The Godfather,"spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,in the continuing saga of the corleone crime f...
3,Schindler's List,the true story of how businessman oskar schind...
4,12 Angry Men,the defense and the prosecution have rested an...


## Removing HTML Tags

In [7]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)
df['Description'] = df['Description'].apply(remove_html_tags)

In [8]:
df.sample(5)

,Name,Description
1490,That Obscure Object of Desire,after dumping a bucket of water on a beautiful...
2193,Missing,based on the real-life experiences of ed horma...
101,Puss in Boots: The Last Wish,puss in boots discovers that his passion for a...
3396,Descendants: The Rise of Red,after the queen of hearts incites a coup on au...
9336,Sorry if I Want to Marry You,"second part of the film ""sorry if i love you""...."


## Removing Links

In [9]:
def removing_links(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    # This pattern is useful to filterout all types of links
    '''
    text1 = 'Check out my notebook https://www.kaggle.com/campusx/notebook8223fc1abb'
    text2 = 'Check out my notebook http://www.kaggle.com/campusx/notebook8223fc1abb'
    text3 = 'Google search here www.google.com'
    text4 = 'For notebook click https://www.kaggle.com/campusx/notebook8223fc1abb to search check www.google.com'
    '''
    return pattern.sub(r'', text)
df['Description'] = df['Description'].apply(removing_links)

In [10]:
df['Description'].sample(5)

,Description
8636,after witnessing a grotesque murder at a party...
4012,a teenager finds himself transported to an isl...
7581,as soon as muffy st. john and her college frie...
6536,"after finding an ad online for “video work,” s..."
8221,max simkin repairs shoes in the same new york ...


In [14]:
!pip install urlextract

In [17]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("Let's have URL www.stackoverflow.com as an example.")
print(urls) # prints: ['stackoverflow.com']

['www.stackoverflow.com']


## Remove Puncuations

In [18]:
import string
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', exclude))# maketrans is a mapping function
df['Description'] = df['Description'].apply(remove_punctuation)

In [20]:
df['Description'][0]

'imprisoned in the 1940s for the double murder of his wife and her lover upstanding banker andy dufresne begins a new life at the shawshank prison where he puts his accounting skills to work for an amoral warden during his long stretch in prison dufresne comes to be admired by the other inmates  including an older prisoner named red  for his integrity and unquenchable sense of hope'

## Chat Words Treatment

In [21]:
chat_words = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'B4N': 'Bye For Now',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'ILU: \'I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A.. Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A..',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick: \'-D Laugher',
    'TFW' : 'That feeling when. TFW internet slang often goes in a caption to an image.',
    'MFW' : 'My face when',
    'MRW' : 'My reaction when',
    'IFYP' : 'I feel your pain',
    'LOL' : 'Laughing out loud',
    'TNTL' : 'Trying not to laugh',
    'JK' : 'Just kidding',
    'IDC' : 'I don’t care',
    'ILY' : 'I love you',
    'IMU' : 'I miss you',
    'ADIH' : 'Another day in hell',
    'IDC' : 'I don’t care',
    'ZZZ' : 'Sleeping, bored, tired',
    'WYWH' : 'Wish you were here',
    'TIME' : 'Tears in my eyes',
    'BAE' : 'Before anyone else',
    'FIMH' : 'Forever in my heart',
    'BSAAW' : 'Big smile and a wink',
    'BWL' : 'Bursting with laughter',
    'LMAO' : 'Laughing my a** off',
    'BFF': 'Best friends forever',
    'CSL' : 'Can’t stop laughing'
}

In [22]:
def chat_words_treatment(text):
    for word in text.split():
        if(word in chat_words):
            text = text.replace(word, chat_words[word])
    return text
df['Description'] = df['Description'].apply(chat_words_treatment)

## Spelling Correction

In [29]:
from textblob import TextBlob
def spelling_correction(text):
    textBlb = TextBlob(text)
    return textBlb.correct().string

spelling_correction("Hello my name is dexter morgan")

'Hello my name is dexter morgan'

## Removing Stop Words

In [36]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word not in stopwords.words('english'):
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

df['Description'] = df['Description'].apply(remove_stopwords)

In [38]:
df.loc[0, 'Description']

'imprisoned 1940s double murder wife lover upstanding banker andy dufresne begins new life shawshank prison puts accounting skills work amoral warden long stretch prison dufresne comes admired inmates including older prisoner named red integrity unquenchable sense hope'

## Removing Emojis

In [39]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Loved the movie. It was 😘😘")

# df['Description'] = df['Description'].apply(remove_emoji)

'Loved the movie. It was '

## Replacing Emojis with there Emotion Text

In [41]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.5 MB/s eta 0:00:00


In [42]:
import emoji
print(emoji.demojize('Python is 🔥'))

Python is :fire:


In [43]:
print(emoji.demojize('Loved the movie. It was 😘'))

Loved the movie. It was :face_blowing_a_kiss:


In [44]:
print(emoji.demojize('Loved the movie. It was 😂'))

Loved the movie. It was :face_with_tears_of_joy:


## Tokenization

In [46]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [47]:
from nltk.tokenize import word_tokenize, sent_tokenize
tokens = word_tokenize("".join(df['Description']))

In [49]:
tokens[:5]

['imprisoned', '1940s', 'double', 'murder', 'wife']

In [51]:
df.loc[0, 'Description']

'imprisoned 1940s double murder wife lover upstanding banker andy dufresne begins new life shawshank prison puts accounting skills work amoral warden long stretch prison dufresne comes admired inmates including older prisoner named red integrity unquenchable sense hope'

In [52]:
sentences_tokens = sent_tokenize("".join(df['Description']))
# Since we removed all the puncuations we got only one token

In [56]:
sent_tokenize("Hello my name is Dexter. I live at Kolhapur")

['Hello my name is Dexter.', 'I live at Kolhapur']

In [57]:
import spacy

# Creating blank language object then
# tokenizing words of the sentence
nlp = spacy.blank("en")

doc = nlp("am am are are")
doc

am am are are

In [58]:
tokens = sent_tokenize("$10 15km 15-20 U.S.")
doc = nlp("$10 15km 15-20 U.S.")

print(tokens)

['$10 15km 15-20 U.S.']


In [59]:
doc

$10 15km 15-20 U.S.

## Lemmatization

In [60]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download WordNet and punkt if you haven't already
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [61]:
# Initialize the lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Sample sentence
sentence = "He was running and eating at the same time. He has a bad habit of swimming after playing long hours in the Sun."

# Tokenize the sentence into words
words = word_tokenize(sentence)

# Lemmatize each word in the sentence with 'v' as the POS tag for verbs
lemmatized_words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

# Join the lemmatized words back into a sentence
lemmatized_sentence = ' '.join(lemmatized_words)

print(lemmatized_sentence)

He be run and eat at the same time . He have a bad habit of swim after play long hours in the Sun .


# Text Representation

In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(lowercase = True, stop_words = 'english', analyzer = 'word', ngram_range = (1, 3), binary = False)
bow = cv.fit_transform(df['Description'])

In [99]:
cv.vocabulary_

{'imprisoned': 191271,
 '1940s': 1389,
 'double': 108876,
 'murder': 258961,
 'wife': 422043,
 'lover': 233240,
 'upstanding': 407478,
 'banker': 30302,
 'andy': 16886,
 'dufresne': 112313,
 'begins': 34362,
 'new': 265888,
 'life': 221885,
 'shawshank': 344919,
 'prison': 299946,
 'puts': 305621,
 'accounting': 6273,
 'skills': 350555,
 'work': 427250,
 'amoral': 16173,
 'warden': 416720,
 'long': 229465,
 'stretch': 367680,
 'comes': 71461,
 'admired': 8057,
 'inmates': 194183,
 'including': 191776,
 'older': 274225,
 'prisoner': 300287,
 'named': 262081,
 'red': 313608,
 'integrity': 195676,
 'unquenchable': 406258,
 'sense': 340372,
 'hope': 183544,
 'imprisoned 1940s': 191274,
 '1940s double': 1392,
 'double murder': 108913,
 'murder wife': 259355,
 'wife lover': 422494,
 'lover upstanding': 233339,
 'upstanding banker': 407479,
 'banker andy': 30305,
 'andy dufresne': 16898,
 'dufresne begins': 112314,
 'begins new': 34645,
 'new life': 266617,
 'life shawshank': 223525,
 'shawsh

In [91]:
len(cv.vocabulary_)

29874

In [92]:
bow[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
encoded = tfidf.fit_transform(df['Description'])

In [101]:
encoded[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [105]:
print(tfidf.idf_)

[9.08343148 9.48889659 8.3902843  ... 9.48889659 9.48889659 9.08343148]


array(['00', '006', '007', ..., 'éric', 'öztürk', 'ʻohana'], dtype=object)

# Deep Learning Approaches

In [116]:
!pip install gensim

In [117]:
from gensim.utils import simple_preprocess

In [118]:
simple_preprocess("Hello My name is Harry")

['hello', 'my', 'name', 'is', 'harry']

In [120]:
store = []
df['Description'].apply(lambda x: store.append(simple_preprocess(x)))

,Description
0,None
1,None
2,None
3,None
4,None
...,...
9715,None
9716,None
9717,None
9718,None


In [122]:
len(store)

9720

In [125]:
import gensim
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    vector_size = 100
)

In [126]:
model.build_vocab(store)

In [130]:
model.epochs

5

In [129]:
model.train(store, total_examples = model.corpus_count, epochs = model.epochs)

(1177118, 1261000)

In [132]:
model.wv.most_similar('boy')

[('meets', 0.9993821382522583),
 ('son', 0.9993306994438171),
 ('mother', 0.9992178678512573),
 ('married', 0.9991076588630676),
 ('couple', 0.9990133047103882),
 ('beautiful', 0.9989761710166931),
 ('whose', 0.9989615678787231),
 ('father', 0.9989044070243835),
 ('daughter', 0.9989011883735657),
 ('teenage', 0.9988861083984375)]

In [133]:
model.wv.doesnt_match(['boy', 'girl', 'man', 'women', 'merry'])

'merry'

In [134]:
model.wv['king']

array([-4.7805241e-01,  8.9421040e-01,  5.9682089e-01,  4.0302407e-02,
       -1.8417187e-01, -1.0727488e+00,  3.2542974e-01,  1.6779027e+00,
       -3.6795244e-01, -2.4228179e-01, -3.5064438e-01, -1.4925445e+00,
       -1.0779732e-02,  3.4555501e-01,  5.9883475e-01, -1.2025588e+00,
        6.4149626e-02, -1.2701110e+00, -3.3447571e-02, -1.7758778e+00,
        4.0661937e-01,  5.2599221e-01,  5.1979232e-01, -2.7209094e-01,
       -3.7241276e-02, -1.4135563e-02, -6.5465808e-01, -4.3948710e-01,
       -9.2859024e-01, -1.2027569e-01,  5.4470801e-01,  1.1613366e-01,
        1.7044647e-03, -5.4109949e-01, -7.6100066e-02,  8.7756443e-01,
        3.2019526e-01, -6.0449409e-01, -3.3756110e-01, -1.6588789e+00,
       -4.8590802e-02, -7.6649451e-01, -4.9502033e-01,  1.7559346e-02,
        5.0756824e-01, -3.2639140e-01, -7.8039438e-01, -5.7014484e-02,
       -1.4682734e-01,  8.7216234e-01, -1.2324012e-01, -6.3025528e-01,
       -7.1349901e-01, -2.1083152e-01, -7.2468585e-01,  7.2306651e-04,
      

In [135]:
model.wv.similarity('bottle','can')

0.99186695

In [136]:
model.wv.get_normed_vectors() # Returns the vector representation of all the words

array([[-0.06355576,  0.13680956,  0.09171266, ..., -0.15839307,
         0.03032753,  0.01129388],
       [-0.08647829,  0.1175487 ,  0.10407068, ..., -0.14106596,
        -0.01215109, -0.0006144 ],
       [-0.05690471,  0.15416141,  0.07277973, ..., -0.15262741,
         0.0419447 ,  0.00231048],
       ...,
       [-0.12808035,  0.18103282,  0.08616749, ..., -0.18451056,
        -0.01988638,  0.00138477],
       [-0.12168589,  0.08864621,  0.03618131, ..., -0.11659244,
        -0.04106048,  0.08991076],
       [-0.09016749,  0.1173208 ,  0.0997637 , ..., -0.14454001,
         0.05217974, -0.01003663]], dtype=float32)

In [139]:
from sklearn.decomposition import PCA

In [140]:
pca = PCA(n_components = 3)

In [141]:
pca.fit_transform(model.wv.get_normed_vectors())

array([[-0.03644544, -0.00485121, -0.03764542],
       [-0.00977886, -0.09970655, -0.10137251],
       [-0.02839023, -0.01952115, -0.08144549],
       ...,
       [ 0.03777266,  0.03631732,  0.04492512],
       [ 0.04612064,  0.02249181,  0.04885054],
       [-0.02260065,  0.03490208, -0.02947539]], dtype=float32)

In [144]:
y = model.wv.index_to_key
y

['life',
 'new',
 'young',
 'one',
 'world',
 'family',
 'two',
 'must',
 'man',
 'find',
 'love',
 'years',
 'finds',
 'friends',
 'woman',
 'home',
 'help',
 'time',
 'back',
 'story',
 'school',
 'lives',
 'soon',
 'get',
 'take',
 'father',
 'group',
 'way',
 'mysterious',
 'war',
 'becomes',
 'wife',
 'town',
 'girl',
 'son',
 'three',
 'takes',
 'city',
 'together',
 'mother',
 'become',
 'first',
 'save',
 'daughter',
 'old',
 'friend',
 'make',
 'team',
 'begins',
 'day',
 'death',
 'discovers',
 'high',
 'secret',
 'police',
 'named',
 'best',
 'gets',
 'past',
 'boy',
 'former',
 'people',
 'set',
 'meets',
 'hes',
 'small',
 'go',
 'forced',
 'york',
 'house',
 'however',
 'american',
 'true',
 'order',
 'mission',
 'night',
 'discover',
 'goes',
 'job',
 'living',
 'falls',
 'work',
 'murder',
 'evil',
 'fight',
 'stop',
 'even',
 'tries',
 'agent',
 'battle',
 'journey',
 'parents',
 'comes',
 'men',
 'film',
 'decides',
 'year',
 'earth',
 'turns',
 'killer',
 'turn',
 'c

In [145]:
import plotly.express as px
fig = px.scatter_3d(store[:100],x=0,y=1,z=2, color=y[:100])
fig.show()